In [1]:
# Go back to parent
import sys
sys.path.append('../..')
print(sys.path)

from src.models.models import ClassifierModels, Embeddings
from src.repo.orm import OpenPool
from src.repo.prayerRequests import PrayerRequestRepoImpl
from src.repo.contacts import ContactRepoImpl
from src.dto.groups import Group
import os
from dotenv import load_dotenv
load_dotenv()
pg_uri = os.environ.get('PRAYERS_PG_DATABASE_URL')

['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\jadog\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\jadog\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32', 'C:\\Users\\jadog\\AppData\\Roaming\\Python\\Python312\\site-packages\\win32\\lib', 'C:\\Users\\jadog\\AppData\\Roaming\\Python\\Python312\\site-packages\\Pythonwin', 'c:\\Program Files\\Python312\\Lib\\site-packages', '../..']


C:\Users\jadog\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Replace Account with the actual class name for the user model
user_id = 1
pool = OpenPool(pg_uri)
embedding_model = Embeddings()
classifier_models = ClassifierModels()
prayerRepo = PrayerRequestRepoImpl(pool, embedding_model, classifier_models)
contactRepo = ContactRepoImpl(pool)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error loading model for prayer type classification
[Errno 2] No such file or directory: 'C:\\Users\\jadog\\Documents\\Code\\Prayer-ML-Tracker\\src\\models\\..\\..\\cache\\prayers\\prayer_or_praise.pkl'


Device set to use cpu
Device set to use cpu
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [18]:
import pandas as pd 
import json

# Load json object, and get the Categories array to use as a dataframe
with open('./Prayermate.json') as f:
    data = json.load(f)
    categories = data['Categories']

prayerRequests = []
for category in categories:
    for subject in category['subjects']:
        for prayerRequest in subject['cards']:
            prayerRequests.append({
                'category': category['name'],
                'subject': subject['name'],
                'prayerRequest': prayerRequest['text'],
                'archivedDate': prayerRequest['archivedDate'] if prayerRequest['archived'] else None,
                'stackGroup': prayerRequest['stackGroup'],
                'createdDate': prayerRequest['createdDate'],
            })

df = pd.DataFrame(prayerRequests)

df['createdDate'] = pd.to_datetime(df['createdDate'])
# df['archivedDate'] = pd.to_datetime(df['archivedDate'])
# replace pd.NaT with None
# df['archivedDate'] = df['archivedDate'].apply(lambda x: None if x is pd.NaT else x)
df[df['subject'] == 'Jadon'].head(2)
# df[df['subject'] == 'Jadon']['archivedDate'].iloc[0] is pd.NaT

,category,subject,prayerRequest,archivedDate,stackGroup,createdDate
475,Monday Bible Study,Jadon,Prayers for Melissa cousin whose wedding got c...,None,ed78b3de-a628-4f17-bfb2-217ed3e3f6df,2023-11-20 19:21:00
476,Monday Bible Study,Jadon,Praise for getting focused over weekend on lea...,None,49c4a9cb-5129-4411-b167-488f44e2a49d,2023-11-20 19:22:00


In [4]:
groups = df['category'].unique()
for group in groups:
    group = Group().from_dict({'name': group})
    contactRepo.save_group(user_id, group)

In [13]:
from src.dto.contacts import Contact
all_groups = contactRepo.get_groups(user_id).to_dataframe()
contacts = df.groupby(['category', 'subject']).agg({
    'createdDate': 'min'
}).reset_index()
for index, row in contacts.iterrows():
    group = all_groups[all_groups['name'] == row['category']]
    group_id = group['id'].values[0]
    contact = Contact().from_dict({'name': row['subject'], 'account_id': user_id, 'created_at': row['createdDate']})
    contactRepo.save_contact(user_id, contact, group_id)

In [4]:
from src.dto.prayerRequests import PrayerRequest, PrayerRequests
from src.dto.link import Link
from src.repo.orm import LinkORM
# Create a dataframe to map stackGroup to a unique integer link_id, order by stackGroup
# Also create a column count to keep track of the number of prayer requests in each stackGroup
stackGroups = df['stackGroup'].unique()
stackGroups = pd.DataFrame(stackGroups, columns=['stackGroup']).sort_values(by='stackGroup')
stackGroups['count'] = df.groupby('stackGroup').size().reset_index(name='counts')['counts']
stackGroups = stackGroups[stackGroups['count'] > 1]
stackGroups.reset_index(inplace=True)
stackGroups['link_id'] = stackGroups.index + 1

In [5]:
with pool() as session:
    for index, row in stackGroups.iterrows():
        link = LinkORM(id=row['link_id'])
        session.add(link)
    session.commit()

In [20]:
from src.dto.prayerRequests import PrayerRequest, PrayerRequests
from src.repo.prayerRequests import PrayerRequestRepoImpl
from src.dto.contacts import Contact
from src.repo.orm import ContactGroupORM, ContactORM, GroupORM
contacts = df.groupby(['category', 'subject']).size().reset_index(name='counts')
with pool() as session:
    for index, row in df.iterrows():
        contactOrm = session.query(ContactGroupORM).filter(ContactORM.name == row['subject'] and GroupORM.name == row['category']).first()
        contact_id = contactOrm.id
        stackGroupRow = stackGroups[stackGroups['stackGroup'] == row['stackGroup']]
        link_id = None if stackGroupRow.empty else int(stackGroupRow['link_id'].values[0])
        prayerRequest = PrayerRequest().from_dict({
            'contact': {'id': int(contact_id)},
            'request': row['prayerRequest'],
            'archived_at': row['archivedDate'],
            'created_at': row['createdDate'],
            'link_id': link_id
        })
        prayerRepo.save(user_id, prayerRequest)